In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import time
import pyautogui

In [ ]:
# Move to mouse 

pyautogui.moveTo(1500,400,duration=1)

pyautogui.FAILSAFE=False

driver_path = "./chromedriver-win64/chromedriver.exe"

main_url = "https://www.hepsiburada.com/"

# Create an empty DataFrame for user reviews and products

try:
    products_df = pd.read_csv('products.csv')
    reviews_df = pd.read_csv('reviews.csv')

except FileNotFoundError:
    products_df = pd.DataFrame(columns=['product url', 'product name', 'img url','brand name','category name'])
    reviews_df = pd.DataFrame(columns=['user reviews', 'rating', 'product url'])
        

def main(num_categories, num_products, category_index):

    global products_df
    global reviews_df
    global start
    global end

    start = time.time()
    service = ChromeService(executable_path=driver_path)
    driver = webdriver.Chrome(service=service)
    driver.get(main_url)

    for j in range(category_index * 2 , category_index * 2 + num_categories*2 ,2):
        # Get the category name

        action = ActionChains(driver)

        navbar_element = driver.find_element(By.CLASS_NAME,'sf-MenuItems-WulWXvlfIAwNiOUGY7FP') 

        driver.execute_script("arguments[0].scrollIntoView();", navbar_element)

        action.move_to_element(navbar_element).perform()

        time.sleep(4)
        
        category_tag = driver.find_elements(By.CLASS_NAME,'sf-ChildMenuItems-a4G0z3YJJWkQs7qKKuuj')[j]

        category_name = category_tag.text

        category_tag.click()

        time.sleep(5)

        # Sort the products from those with the most comments to fewest 

        order_button = driver.find_elements(By.CSS_SELECTOR,"div[data-test-id='dropdown-container']")[1]

        driver.execute_script("arguments[0].scrollIntoView();", order_button)

        order_button.click()

        time.sleep(5)

        sort_by_reviews = driver.find_elements(By.CLASS_NAME,'horizontalSortingBar-PkoDOH7UsCwBrQaQx9bn')

        sort_by_reviews[3].click()

        driver.refresh()

        # Find <li> tags containing the list of products

        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_"))
        )

        li_tags = driver.find_elements(By.CLASS_NAME,"productListContent-zAP0Y5msy8OHn5z7T_K_")

        # Loop through each product

        product_counter = 0 

        for li in li_tags[0 : num_products]:
            product_url = None
            product_name = None
            img_url = None
            user_review = None
            rating = None
            brand_name = None
            
            # Get the product URL
            product_url = li.find_element(By.TAG_NAME,"a").get_attribute("href")

            if product_url not in products_df['product url'].tolist():
                    
                # Open a new window, Switch to the new window
                driver.execute_script("window.open('');")

                driver.switch_to.window(driver.window_handles[-1]) 

                driver.get(product_url)

                driver.set_window_size(1200, driver.get_window_size()["height"])

                # Get the product name
                product_name = driver.find_element(By.ID,"product-name").text

                # Get the brand name 
                brand_name = driver.find_element(By.XPATH,"/html/body/div[2]/main/div[3]/section[1]/div[3]/div/div[4]/div[1]/span/a").text

                # Get the product image URL
                img_url = driver.find_element(By.CSS_SELECTOR,"img[class='product-image']").get_attribute("src")

                # Add products to the DataFrame
                products_df.loc[len(products_df)]=[product_url,product_name,img_url,brand_name,category_name]

                # Scroll down the page, Wait for user reviews to load
                driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                driver.find_element(By.XPATH,'//*[@id="productReviewsTab"]').click()

                # Process each user review
                try:
                    all_reviews_button = WebDriverWait(driver, 10).until(
                            EC.presence_of_element_located((By.XPATH, "//*[@id='hermes-voltran-comments']/div[2]/div[1]/a"))).click()
                    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                    try:
                        last_reviews = driver.find_elements(By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/ul/li')[-1].text
                        
                    except IndexError:
                        last_reviews = 1

                    driver.set_window_size(400, driver.get_window_size()["height"])

                    #for i in range(int(last_reviews)):
                    i = 0
                    
                    while i < int(last_reviews):

                        next_page_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(
                            (By.XPATH,'//*[@id="hermes-voltran-comments"]/div[6]/div[3]/div[2]/div/div/div/div[3]')))
                        
                        driver.execute_script("arguments[0].scrollIntoView();", next_page_button)

                        try:
                            all_reviews_div = WebDriverWait(driver, 5).until(
                                EC.presence_of_all_elements_located((By.CLASS_NAME,"hermes-ReviewCard-module-BJtQZy5Ub3goN_D0yNOP")))

                            WebDriverWait(driver, 5).until(
                                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[class='star']")))

                            WebDriverWait(driver, 5).until(
                                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "span[itemprop='description']")))

                        except:
                            pass

                        counter = 0

                        for review in all_reviews_div:
                            try:
                                rating = len(review.find_elements(By.CSS_SELECTOR, "div[class='star']"))
                            except:
                                rating = None
                            try:
                                user_review = review.find_element(By.CSS_SELECTOR, "span[itemprop='description']").text

                            except:
                                #user_review = None
                                # Counter to check review without text
                                counter += 1
                                if counter == 3:
                                    i = int(last_reviews)
                                    break

                            # Add products to the DataFrame   
                            reviews_df.loc[len(reviews_df)] = [user_review,rating,product_url]

                        next_page_button.click()

                        i+=3
                        
                    driver.close()

                    # Switch back to the main window
                    driver.switch_to.window(driver.window_handles[0])

                    product_counter += 1
                    print(f'Produt {product_counter} taken')

                except TimeoutException:
                    driver.close()
                    
                    # Switch back to the main window
                    driver.switch_to.window(driver.window_handles[0])
                    continue

        driver.set_window_size(1200, driver.get_window_size()["height"])
    
    driver.quit()
    end = time.time()

In [ ]:
# num_categories is number of category
# num_products is number of products
# category_index is index of category
main(num_categories=2, num_products=5, category_index=2)

In [ ]:
duration_minutes = (end - start) / 60
print(f"Execution time: {int(duration_minutes)} minutes")

In [ ]:
#number of reviews for each category
sum_rev = []
rev_name = []
for i in products_df['category name'].unique():
    temp_list = []

    for a in products_df[products_df['category name'] == i]['product url']:
        temp_list.append(len(reviews_df[reviews_df['product url'] == a]))
    sum_rev.append(temp_list)
    rev_name.append(i)

for a, b in zip(sum_rev, rev_name):
    print(a , ':', b)

In [ ]:
reviews_df

In [ ]:
products_df

In [ ]:

reviews_df[reviews_df["product url"]==a]

In [ ]:
reviews_df[reviews_df[["user reviews","rating"]].isna()]

In [ ]:
len(reviews_df["user reviews"].unique())

In [ ]:
len(reviews_df[reviews_df["user reviews"].isna()])

In [ ]:
products_df.to_csv('products_df.csv',index=False,encoding="utf-8")

In [ ]:
reviews_df.to_csv('reviews_df.csv',index=False,encoding="utf-8")